In [14]:
import yfinance as yf
import pandas as pd
symbol = 'AAPL'
df = yf.download(symbol, start='2022-01-01', proxy="127.0.0.1:33210")
# print(df)
import math
# https://www.investopedia.com/articles/investing/102715/computing-historical-volatility-excel.asp
# 用上面链接，根据前一段时间的股价计算volatility
df.insert(df.shape[1],'Ln',0)
df.insert(df.shape[1],'Ln2',0)
for i in range(df.shape[1]):
    df['Ln'].iloc[i+1] = math.log(df['Close'].iloc[i+1]/df['Close'].iloc[i])
    df['Ln2'].iloc[i+1] = df['Ln'].iloc[i+1]**2
volatility_daily = math.sqrt(df['Ln2'].sum()/(df.shape[1]-1-1))
print('volatility is',volatility_daily)

#用历史数据法计算5%和1%的value at risk
import numpy as np
df['daily_ret'] = df['Close'].pct_change(1)
# print(df)
sRate = df['daily_ret'].iloc[1:].sort_values(ascending=True)
print(sRate)
p = np.percentile(sRate, (1, 5, 10), interpolation='midpoint') # 输出分位度为1%，5%和10%即置信度99%，95%和90%时的值
print(p) #1%分位值为第一个-0.05377872，即根据历史数据，value at risk回报率高于-0.05377872的可能为99%


[*********************100%***********************]  1 of 1 completed
volatility is 0.0185648603506255
Date
2022-05-18   -0.056419
2022-05-05   -0.055716
2022-05-11   -0.051841
2022-06-16   -0.039651
2022-06-10   -0.038629
                ...   
2022-05-23    0.040119
2022-05-27    0.040757
2022-05-04    0.041008
2022-04-28    0.045155
2022-01-28    0.069778
Name: daily_ret, Length: 134, dtype: float64
[-0.05377872 -0.03780647 -0.02750371]


C:\Users\szf1998\AppData\Local\Temp\ipykernel_20044\1960746959.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Ln'].iloc[i+1] = math.log(df['Close'].iloc[i+1]/df['Close'].iloc[i])
C:\Users\szf1998\AppData\Local\Temp\ipykernel_20044\1960746959.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Ln2'].iloc[i+1] = df['Ln'].iloc[i+1]**2
C:\Users\szf1998\AppData\Local\Temp\ipykernel_20044\1960746959.py:23: DeprecationWarning: the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review

In [38]:
import yfinance as yf
import pandas as pd
symbol = 'AAPL'
df = yf.download(symbol, start='2020-01-01', proxy="127.0.0.1:33210")
print(df)
from ta.trend import MACD
from ta.momentum import StochasticOscillator
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=4, cols=1, shared_xaxes=True)
fig.add_trace(go.Candlestick(x=df.index,
  open=df['Open'],
  high=df['High'],
  low=df['Low'],
  close=df['Close'],
  showlegend=False), row=1, col=1)

#把非交易日排除掉
# hide weekends
fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])]) 
# removing all empty dates
# build complete timeline from start date to end date
dt_all = pd.date_range(start=df.index[0],end=df.index[-1])
# retrieve the dates that ARE in the original datset
dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(df.index)]
# define dates with missing values
dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]
fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

#add moving average
df['MA20'] = df['Close'].rolling(window=20).mean()
df['MA5'] = df['Close'].rolling(window=5).mean()
df['MA50'] = df['Close'].rolling(window=50).mean()
fig.add_trace(go.Scatter(x=df.index, 
                         y=df['MA5'], 
                         opacity=0.7, 
                         line=dict(color='blue', width=2), 
                         name='MA 5'), row=1, col=1)

fig.add_trace(go.Scatter(x=df.index, 
                         y=df['MA20'], 
                         opacity=0.7, 
                         line=dict(color='orange', width=2), 
                         name='MA 20'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, 
                         y=df['MA50'], 
                         opacity=0.7, 
                         line=dict(color='pink', width=2), 
                         name='MA 50'), row=1, col=1)


# MACD
macd = MACD(close=df['Close'], 
            window_slow=26,
            window_fast=12, 
            window_sign=9)
# stochastic
stoch = StochasticOscillator(high=df['High'],
                             close=df['Close'],
                             low=df['Low'],
                             window=14, 
                             smooth_window=3)


# Plot OHLC on 1st subplot (using the codes from before)
# Plot volume trace on 2nd row 
fig.add_trace(go.Bar(x=df.index, 
                     y=df['Volume']
                    ), row=2, col=1)
# Plot MACD trace on 3rd row
fig.add_trace(go.Bar(x=df.index, 
                     y=macd.macd_diff()
                    ), row=3, col=1)
fig.add_trace(go.Scatter(x=df.index,
                         y=macd.macd(),
                         line=dict(color='black', width=2)
                        ), row=3, col=1)
fig.add_trace(go.Scatter(x=df.index,
                         y=macd.macd_signal(),
                         line=dict(color='blue', width=1)
                        ), row=3, col=1)
# Plot stochastics trace on 4th row
fig.add_trace(go.Scatter(x=df.index,
                         y=stoch.stoch(),
                         line=dict(color='black', width=2)
                        ), row=4, col=1)
fig.add_trace(go.Scatter(x=df.index,
                         y=stoch.stoch_signal(),
                         line=dict(color='blue', width=1)
                        ), row=4, col=1)
#传统的stochastics oscillator 80超买，20超卖,把这个加进stochastics
import numpy as np
const1 = np.linspace(start=20, stop=20,num=len(df.index))
const2 = np.linspace(start=80, stop=80,num=len(df.index))
fig.add_trace(go.Scatter(x=df.index,
                         y=const1,
                         line=dict(color='red',width=1)
                        ),row=4,col=1
                 )
fig.add_trace(go.Scatter(x=df.index,
                         y=const2,
                         line=dict(color='green',width=1)
                        ),row=4,col=1
                 )
# update layout by changing the plot size, hiding legends & rangeslider, and removing gaps between dates
fig.update_layout(height=900, width=1200, 
                  showlegend=False, 
                  xaxis_rangeslider_visible=False,
                  xaxis_rangebreaks=[dict(values=dt_breaks)])


fig.show()
fig.write_image('plotly_test1.jpeg')



[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2019-12-31   72.482498   73.419998   72.379997   73.412498   72.139954   
2020-01-02   74.059998   75.150002   73.797501   75.087502   73.785912   
2020-01-03   74.287498   75.144997   74.125000   74.357498   73.068558   
2020-01-06   73.447502   74.989998   73.187500   74.949997   73.650795   
2020-01-07   74.959999   75.224998   74.370003   74.597504   73.304420   
...                ...         ...         ...         ...         ...   
2022-07-12  145.759995  148.449997  145.050003  145.860001  145.860001   
2022-07-13  142.990005  146.449997  142.119995  145.490005  145.490005   
2022-07-14  144.080002  148.949997  143.250000  148.470001  148.470001   
2022-07-15  149.779999  150.860001  148.199997  150.169998  150.169998   
2022-07-18  150.740005  151.570007  146.699

In [36]:
# 单独看macd
import yfinance as yf
import pandas as pd
symbol = 'AAPL'
df = yf.download(symbol, start='2020-01-01', proxy="127.0.0.1:33210")
print(df)
from ta.trend import MACD
from ta.momentum import StochasticOscillator
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = go.Figure(go.Candlestick(x=df.index,
  open=df['Open'],
  high=df['High'],
  low=df['Low'],
  close=df['Close'],
  showlegend=False))



#add moving average
df['MA20'] = df['Close'].rolling(window=20).mean()
df['MA5'] = df['Close'].rolling(window=5).mean()
df['MA50'] = df['Close'].rolling(window=50).mean()
fig.add_trace(go.Scatter(x=df.index, 
                         y=df['MA5'], 
                         opacity=0.7, 
                         line=dict(color='blue', width=2), 
                         name='MA 5'))

fig.add_trace(go.Scatter(x=df.index, 
                         y=df['MA20'], 
                         opacity=0.7, 
                         line=dict(color='orange', width=2), 
                         name='MA 20'))
fig.add_trace(go.Scatter(x=df.index, 
                         y=df['MA50'], 
                         opacity=0.7, 
                         line=dict(color='pink', width=2), 
                         name='MA 50'))
#把非交易日排除掉
# hide weekends
fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])]) 
# removing all empty dates
# build complete timeline from start date to end date
dt_all = pd.date_range(start=df.index[0],end=df.index[-1])
# retrieve the dates that ARE in the original datset
dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(df.index)]
# define dates with missing values
dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]
fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])
fig.show()

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2019-12-31   72.482498   73.419998   72.379997   73.412498   72.139954   
2020-01-02   74.059998   75.150002   73.797501   75.087502   73.785912   
2020-01-03   74.287498   75.144997   74.125000   74.357498   73.068565   
2020-01-06   73.447502   74.989998   73.187500   74.949997   73.650795   
2020-01-07   74.959999   75.224998   74.370003   74.597504   73.304405   
...                ...         ...         ...         ...         ...   
2022-07-08  145.259995  147.550003  145.000000  147.039993  147.039993   
2022-07-11  145.669998  146.639999  143.779999  144.869995  144.869995   
2022-07-12  145.759995  148.449997  145.050003  145.860001  145.860001   
2022-07-13  142.990005  146.449997  142.119995  145.490005  145.490005   
2022-07-14  144.080002  148.949997  143.250

In [36]:
#单独看Stochastic
import yfinance as yf
import pandas as pd
symbol = 'TSLA'
df = yf.download(symbol, start='2020-01-01', proxy="127.0.0.1:33210")
print(df)
from ta.trend import MACD
from ta.momentum import StochasticOscillator
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = go.Figure(go.Candlestick(x=df.index,
  open=df['Open'],
  high=df['High'],
  low=df['Low'],
  close=df['Close'],
  showlegend=False))

# stochastic
stoch = StochasticOscillator(high=df['High'],
                             close=df['Close'],
                             low=df['Low'],
                             window=14, 
                             smooth_window=3)

fig.add_trace(go.Scatter(x=df.index,
                         y=stoch.stoch(),
                         line=dict(color='black', width=2)
                        ))
fig.add_trace(go.Scatter(x=df.index,
                         y=stoch.stoch_signal(),
                         line=dict(color='blue', width=1)
                        ))
print(df.index)
import numpy as np
const1 = np.linspace(start=20, stop=20,num=len(df.index))
fig.add_trace(go.Scatter(x=df.index,
                         y=const1,
                         line=dict(color='orange',width=1)
                        ))
# fig.add_trace(go.Scatter(x=df.index,
#                          y=80,
#                          line=dict(color='orange',width=1)
#                         ))

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2019-12-31   81.000000   84.258003   80.416000   83.666000   83.666000   
2020-01-02   84.900002   86.139999   84.342003   86.052002   86.052002   
2020-01-03   88.099998   90.800003   87.384003   88.601997   88.601997   
2020-01-06   88.094002   90.311996   88.000000   90.307999   90.307999   
2020-01-07   92.279999   94.325996   90.671997   93.811996   93.811996   
...                ...         ...         ...         ...         ...   
2022-07-12  710.539978  719.320007  685.109985  699.210022  699.210022   
2022-07-13  676.500000  726.179993  675.099976  711.119995  711.119995   
2022-07-14  704.690002  715.960022  688.000000  714.940002  714.940002   
2022-07-15  720.000000  730.869995  710.669983  720.200012  720.200012   
2022-07-18  734.809998  751.549988  718.809

In [31]:
import numpy as np
a = np.ones[20]*80

TypeError: 'function' object is not subscriptable